In [4]:
import mysql.connector
import pandas as pd

In [6]:
try:
    db_conn = mysql.connector.connect(host = "34.69.145.125", user = "rentechuser", password = "Agdj8ekee9u04IIid", database = "rentech_db")
    query = "SELECT * FROM `mtnewswire_raw_data_test`;"
    sql_query = pd.read_sql(query, db_conn)
    df = pd.DataFrame(sql_query, columns = ['file_name','revision_no','title','body','publish_date','date_string','images','news_type','master_id','job_id'])
except Exception as e:
    print(f'Mysql Error',e)
finally:
    if(db_conn.is_connected()):
        db_conn.close()
        print('connection is closed')
df = df[df['file_name'].str.startswith('G')]

connection is closed


In [7]:
columns = ['file_name','revision_no','title','body','publish_date','date_string','images','news_type','master_id','job_id']
temp_df = df[columns]
temp_df['file_name'] = temp_df['file_name'].apply(lambda x: x.replace('_0','').replace('_1','') if type(x)== str else x)

In [8]:
df2 = temp_df[['file_name','revision_no']].drop_duplicates()
map_dict = df2['file_name'].value_counts().to_dict()
df2['file_count'] = df2['file_name'].map(map_dict)

In [9]:
file_names = df2[df2['file_count'] > 1]['file_name']
file_names = file_names[0:50]

In [14]:
file_names

34       G2078818
81       G2056867
168      G2065952
371      G2076449
506      G2058704
552      G2065769
738      G2056771
1777     G2058179
2429     G2069317
2891     G2058693
2998     G2078310
3128     G2078268
3251     G2076010
3783     G2073055
4518     G2067581
4601     G2064593
5581     G2068852
5594     G2067143
5851     G2065430
5873     G2069286
6680     G2082180
7204     G2058860
7340     G2062987
8121     G2074500
8226     G2066047
8395     G2081325
8894     G2061020
9123     G2071253
9145     G2081112
9457     G2070506
9751     G2071462
9826     G2081120
10164    G2079553
10272    G2062086
10453    G2079387
10608    G2064922
10752    G2056924
10867    G2076710
11026    G2071985
11241    G2060278
11339    G2062185
11453    G2074229
11934    G2060779
12409    G2065812
12499    G2082406
12631    G2056819
13159    G2071996
13290    G2071750
13781    G2062051
14026    G2060913
Name: file_name, dtype: object

In [10]:
try:
    db_conn = mysql.connector.connect(host = "34.69.145.125", user = "rentechuser", password = "Agdj8ekee9u04IIid", database = "rentech_db")
    cursor = db_conn.cursor()
    for file in file_names[3:]:
        sql = f"DELETE FROM `mtnewswire_raw_data` WHERE `file_name` LIKE '%{file}%';"
        cursor.execute(sql)
    db_conn.commit()
    # print(f'{cursor.rowcount} records are inserted successfully')
except Exception as e:
    print(f'Mysql Error',e)
finally:
    if(db_conn.is_connected()):
        cursor.close()
        db_conn.close()
        print('connection is closed')

connection is closed


In [11]:
df3 = temp_df[temp_df['file_name'].isin(file_names)]
df3['file_name_rev'] = df3['file_name']+'_'+df3['revision_no'].astype(str)

C:\Users\011\AppData\Local\Temp/ipykernel_11696/240811699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['file_name_rev'] = df3['file_name']+'_'+df3['revision_no'].astype(str)


In [12]:
resultset = []
for i, row in df3.iterrows():
    resultset.append([row[10],row[1],row[2],row[3],row[4],row[5],row[6],row[8],row[9]])

In [13]:
try:
    db_conn = mysql.connector.connect(host = "34.69.145.125", user = "rentechuser", password = "Agdj8ekee9u04IIid", database = "rentech_db")
    cursor = db_conn.cursor()
    sql = "INSERT IGNORE INTO `mtnewswire_raw_data` VALUES(NULL, %s, %s, %s, %s, %s, %s, %s, 100, %s, %s, CURRENT_TIMESTAMP());"
    cursor.executemany(sql, resultset)
    db_conn.commit()
    print(f'{cursor.rowcount} records are inserted successfully')
except Exception as e:
    print(f'Mysql Error',e)
finally:
    if(db_conn.is_connected()):
        cursor.close()
        db_conn.close()
        print('connection is closed')

106 records are inserted successfully
connection is closed
